# Portfolio optimisation
This notebook illustrates an application of ORCA's PT-Series to portfolio optimisation, where ORCA's QUBO optimiser is used to decide
which companies (with discrete weights) should be included in a portfolio to maximise return and minimise risk.

We start by performing the required imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

if os.getcwd()[-12:]=="optimization":
    os.chdir("../..")

from quantumqubo.qubo import QUBO
from applications_notebooks.portfolio_optimization.utils_finance import get_returns_and_cov, decode_bits_sequence, plot_random_portfolio


### The problem

We want to find the optimal proportions of investment $\omega$ between $N_a$ companies, $\omega \in [0,1]^{N_a}$. It can be formulated as minimizing:

$E = -\underbrace{\omega^T\mu}_{return} + \gamma \underbrace{\omega^T \Sigma \omega}_{risk}$  $\quad$ with    $||\omega|| = 1 $

$\gamma$ is the risk adversion, $\mu$ is the return (vector) and $\Sigma$ is the covariance matrix.

In [ ]:
N = 20 # Number of companies
Nq = 1 # Number of bits used to discretise company weight in a portfolio. A value of 1 means the decision is binary and all companies have equal weight
gamma = 5 # Penalty for the variance


### Load stock market data for some number of companies, and determine average returns and covariance matrix
We use stock market data between 2010 and 2017 for up to 60 companies with the following tickers to determine average historical returns and covariance.

In [ ]:
companies = ['AAPL', 'TSLA', 'AMZN', 'TSM', 'JPM', 'BABA', 'JNJ', 'UNH', 'WMT', 'V', 'BAC', 'HD', 
             'PG', 'MA', 'CRM', 'NKE', 'NVO', 'XOM', 'ORCL', 'TM', 'TMO', 'PFE', 'LLY', 'KO', 'ACN',
             'ABT', 'MRK', 'DHR', 'CVX', 'VZ', 'WFC', 'ECL', 'MS', 'CL', 'NVS', 'SHOP', 'MCD', 'UPS', 
             'SAP', 'T', 'LIN', 'NEE', 'MDT', 'LOW', 'UNP', 'HON', 'RY', 'SCHW', 'SONY', 'PM', 'BLK', 
             'UL', 'GS', 'C', 'EL', 'CVS', 'GE', 'SQ', 'CAT', 'APD' ]

companies = companies[:N]

mu, Sigma = get_returns_and_cov(
    companies,
    start='2010-1-1', 
    end='2017-1-1'
)

## Initialize and run the QUBO solver
We first define the function to minimise, then initialize and run our QUBO solver

In [ ]:
def portfolio_function(bit_string):
    
    weights = decode_bits_sequence(bit_string, Nq)
    
    if np.sum(weights) == 0:
        return 100
    
    weights = weights/np.sum(weights)
  
    return0 = np.dot(weights, mu)
    risk0 = np.dot(weights, np.dot(Sigma, weights))
    
    return -return0 + gamma*risk0

In [ ]:
M = N*Nq  # Total number of photons

qubo = QUBO(
    M,
    portfolio_function
)

qubo.train(
    learning_rate=1e-1,
    updates=100,
    samples_per_point=100,
    print_frequency=20
    )

## Investigate results

In [ ]:
index = 0    # min value of qubo.res
vects = list(qubo.res.keys())
weights = decode_bits_sequence(vects[index], Nq)
weights = weights/np.sum(weights)

final_portfolio = {}
for idx, weight in enumerate(weights):
    if weight > 0:
        final_portfolio[companies[idx]] = weight

print("The optimal portfolio weights found by QUBO with these settings is")
pprint(final_portfolio)

In [ ]:
return1 = np.dot(weights, mu)
risk1 = np.dot(weights, np.dot(Sigma, weights))
print("This portfolio has an average annual return of {:.2f}, annual standard deviation of {:.2f}, and a Sharpe ratio of {:.2f}"\
     .format(return1, np.sqrt(risk1), return1/np.sqrt(risk1)))

# Plot return vs risk for different portfolios
We aim to be on the "efficient frontier", which is the portfolio with the highest return for a given risk. On this graph, blue points represent randomly selected portfolios, and the orange point represents the portfolio found by our algorithm. For a given level of risk (the standard deviation), we aim to have the highest historical return.

In [ ]:
np.random.seed(2)

returns = []
volatilities = []
N=len(companies)

for x in range(10000):
    bit_string = np.random.randint(0,2,N)
    weights = bit_string/np.sum(bit_string)
    returns.append(np.sum(weights * mu))  # annual basis
    volatilities.append(np.dot(weights.T, np.dot(Sigma, weights)))

plt.figure(figsize=(10,6))
plt.plot(np.sqrt(volatilities), returns, 'o', alpha=0.12, label='Random portfolio')
plt.plot([np.sqrt(risk1)],[return1], 'o', label='Our solutions', color="orange")
plt.xlabel('Annual Standard Deviation')
plt.ylabel('Annual Return')
plt.legend()
plt.show()